# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources (README.md file)
- Happy learning!

In [59]:
# import numpy and pandas
import numpy as np
import pandas as pd


# Challenge 1 - Exploring the Data

In this challenge, we will examine all salaries of employees of the City of Chicago. We will start by loading the dataset and examining its contents.

In [60]:
# Your code here:

salaries = pd.read_csv('Current_Employee_Names__Salaries__and_Position_Titles.csv')


Examine the `salaries` dataset using the `head` function below.

In [61]:
# Your code here:
salaries.head()

,Name,Job Titles,Department,Full or Part-Time,Salary or Hourly,Typical Hours,Annual Salary,Hourly Rate
0,"AARON, JEFFERY M",SERGEANT,POLICE,F,Salary,NaN,101442.0,NaN
1,"AARON, KARINA",POLICE OFFICER (ASSIGNED AS DETECTIVE),POLICE,F,Salary,NaN,94122.0,NaN
2,"AARON, KIMBERLEI R",CHIEF CONTRACT EXPEDITER,GENERAL SERVICES,F,Salary,NaN,101592.0,NaN
3,"ABAD JR, VICENTE M",CIVIL ENGINEER IV,WATER MGMNT,F,Salary,NaN,110064.0,NaN
4,"ABASCAL, REECE E",TRAFFIC CONTROL AIDE-HOURLY,OEMC,P,Hourly,20.0,NaN,19.86


We see from looking at the `head` function that there is quite a bit of missing data. Let's examine how much missing data is in each column. Produce this output in the cell below

In [62]:
# Your code here:
salaries.isna().sum()

Name                     0
Job Titles               0
Department               0
Full or Part-Time        0
Salary or Hourly         0
Typical Hours        25161
Annual Salary         8022
Hourly Rate          25161
dtype: int64

Let's also look at the count of hourly vs. salaried employees. Write the code in the cell below

In [63]:
# Your code here:
sal = salaries.loc[salaries['Salary or Hourly'] == 'Salary', 'Salary or Hourly'].value_counts()
hour = salaries.loc[salaries['Salary or Hourly'] == 'Hourly', 'Salary or Hourly'].value_counts()

#or

salaries['Salary or Hourly'].value_counts()



Salary    25161
Hourly     8022
Name: Salary or Hourly, dtype: int64

What this information indicates is that the table contains information about two types of employees - salaried and hourly. Some columns apply only to one type of employee while other columns only apply to another kind. This is why there are so many missing values. Therefore, we will not do anything to handle the missing values.

There are different departments in the city. List all departments and the count of employees in each department.

In [64]:
# Your code here:

depart = salaries.groupby(['Department']).agg({'Name':'count'})
depart

,Name
Department,
ADMIN HEARNG,39
ANIMAL CONTRL,81
AVIATION,1629
BOARD OF ELECTION,107
BOARD OF ETHICS,8
BUDGET & MGMT,46
BUILDINGS,269
BUSINESS AFFAIRS,171
CITY CLERK,84


# Challenge 2 - Hypothesis Tests

In this section of the lab, we will test whether the hourly wage of all hourly workers is significantly different from $30/hr. Import the correct one sample test function from scipy and perform the hypothesis test for a 95% two sided confidence interval.

In [65]:
# Your code here:

# H0 :  hourly wage of all hourly workers = $30/hr
# h1 : !=

from scipy.stats import ttest_1samp, ttest_rel, ttest_ind
from scipy.stats import f_oneway

samp = salaries['Hourly Rate'].dropna()

t_test = ttest_1samp(samp, 30)
p_value = t_test[1]
a = 1 - 0.95

a > p_value

#We reject the null hypotesis. The hourly wage for all hourly workers is significantly different from $30/h

True

We are also curious about salaries in the police force. The chief of police in Chicago claimed in a press briefing that salaries this year are higher than last year's mean of $86000/year a year for all salaried employees. Test this one sided hypothesis using a 95% confidence interval.

Hint: A one tailed test has a p-value that is half of the two tailed p-value. If our hypothesis is greater than, then to reject, the test statistic must also be positive.

In [97]:
# Your code here:

pol_salaries = salaries.loc[salaries['Department'] == 'POLICE']

year_annual_sal_for_pol = pol_salaries['Annual Salary'].dropna()


ttest = ttest_1samp(year_annual_sal_for_pol, 86000)

p_value = ttest[1]
a = (1 - 0.95)/2

a > p_value

True

Using the `crosstab` function, find the department that has the most hourly workers. 

In [98]:
# Your code here:
tab = pd.crosstab(salaries['Department'],salaries['Hourly Rate'])

tab.sum(axis = 1).sort_values(ascending=False)


Department
STREETS & SAN            1862
WATER MGMNT              1513
OEMC                     1273
AVIATION                 1082
GENERAL SERVICES          765
TRANSPORTN                725
PUBLIC LIBRARY            299
FAMILY & SUPPORT          287
CITY COUNCIL               64
FINANCE                    44
LAW                        40
ANIMAL CONTRL              19
POLICE                     10
MAYOR'S OFFICE              8
CULTURAL AFFAIRS            7
BUSINESS AFFAIRS            7
COMMUNITY DEVELOPMENT       4
HUMAN RESOURCES             4
HEALTH                      3
FIRE                        2
PROCUREMENT                 2
BUDGET & MGMT               2
dtype: int64

### The workers from the department with the most hourly workers have complained that their hourly wage is less than $35/hour. Using a one sample t-test, test this one-sided hypothesis at the 95% confidence level.

In [99]:
# Your code here:

STREETS_salaries = salaries.loc[salaries['Department'] == 'STREETS & SAN']

Hourly_wage = pol_salaries['Hourly Rate'].dropna()


ttest = ttest_1samp(Hourly_wage, 35)

p_value = ttest[1]
a = (1 - 0.95)/2

a > p_value



True

# Challenge 3: To practice - Constructing Confidence Intervals

While testing our hypothesis is a great way to gather empirical evidence for accepting or rejecting the hypothesis, another way to gather evidence is by creating a confidence interval. A confidence interval gives us information about the true mean of the population. So for a 95% confidence interval, we are 95% sure that the mean of the population is within the confidence interval. 
).

To read more about confidence intervals, click [here](https://en.wikipedia.org/wiki/Confidence_interval).


In the cell below, we will construct a 95% confidence interval for the mean hourly wage of all hourly workers. 

The confidence interval is computed in SciPy using the `t.interval` function. You can read more about this function [here](https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.stats.t.html).

To compute the confidence interval of the hourly wage, use the 0.95 for the confidence level, number of rows - 1 for degrees of freedom, the mean of the sample for the location parameter and the standard error for the scale. The standard error can be computed using [this](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.sem.html) function in SciPy.

In [110]:
# Your code here:
from scipy.stats import sem
from scipy.stats import t


#calculate mean for loc in intervale

mean = np.mean(Hourly_wage)

#calculate standard error with scipy.stats.sem - for using as scale in the intervale  

sderror = sem(Hourly_wage, ddof=1) ## WHY NOT ddof=len(year_annual_sal_for_pol)-1 

#interval(alpha, df, loc=0, scale=1)

t.interval(0.95, len(Hourly_wage)-1 , loc = mean, scale = sderror) # WHY df paramether ir not an array but the degrees of freedom for interval method ?

(9.31381234362183, 9.45418765637817)

Now construct the 95% confidence interval for all salaried employeed in the police in the cell below.

In [101]:
# Your code here:

mean = np.mean(year_annual_sal_for_pol)

sal_employee_pol = sem(year_annual_sal_for_pol, ddof=1)


t.interval(0.95, len(year_annual_sal_for_pol)-1 , loc = mean, scale = sderror) 

(86486.35368609338, 86486.4753201734)

# Bonus Challenge - Hypothesis Tests of Proportions

Another type of one sample test is a hypothesis test of proportions. In this test, we examine whether the proportion of a group in our sample is significantly different than a fraction. 

You can read more about one sample proportion tests [here](http://sphweb.bumc.bu.edu/otlt/MPH-Modules/BS/SAS/SAS6-CategoricalData/SAS6-CategoricalData2.html).

In the cell below, use the `proportions_ztest` function from `statsmodels` to perform a hypothesis test that will determine whether the number of hourly workers in the City of Chicago is significantly different from 25% at the 95% confidence level.

In [102]:
# Your code here:

from statsmodels.stats.proportion import proportions_ztest



In [126]:
chicago_workers = len(salaries['Hourly Rate'])
hourlyworkers = len(salaries['Hourly Rate'].dropna())
hourlyworkers

8022

In [ ]:
#statsmodels.stats.proportion.proportions_ztest(count, nobs, value=None, alternative='two-sided', prop_var=False)

In [129]:
proportions_ztest(hourlyworkers, chicago_workers,value=0.25)

p_value = proportions_ztest(hourlyworkers, chicago_workers,value=0.25)[1]

a = (1 - 0.95)

a > p_value




True

In [ ]:
"""
null hipotesis : the number of hourly works is equal to 25%
alternative hipotesis : the number of hourly works is significantly diferent from 25%


We reject the null hipotesis because the p_value is lower that our significance level of 0.05%
wich means that we accept our alternative hipotesisis 
"""